# CYP P450 2C9 Inhibition, Veith et al.

### Dataset Description: The CYP P450 genes are involved in the formation and breakdown (metabolism) of various molecules and chemicals within cells. Specifically, the CYP P450 2C9 plays a major role in the oxidation of both xenobiotic and endogenous compounds.

### Task Description: Binary Classification. Given a drug SMILES string, predict CYP2C9 inhibition.

### Dataset Statistics: 12,092 drugs.

### Metric: AUPRC

## Leaderboard

| Rank | Model                                    | Contact         | Link          | #Params   | AUPRC         |
|------|------------------------------------------|-----------------|---------------|-----------|---------------|
| 1    | MapLight + GNN                           | Jim Notwell     | GitHub, Paper | N/A       | 0.859 ± 0.001 |
| 2    | ContextPred                              | Kexin Huang     | GitHub, Paper | 2,067,053 | 0.839 ± 0.003 |
| 3    | AttrMasking                              | Kexin Huang     | GitHub, Paper | 2,067,053 | 0.829 ± 0.003 |
| 4    | ZairaChem                                | Gemma Turon     | GitHub, Paper | N/A       | 0.786 ± 0.004 |
| 5    | MapLight                                 | Jim Notwell     | GitHub, Paper | N/A       | 0.783 ± 0.002 |
| 6    | Chemprop-RDKit                           | Kyle Swanson    | GitHub, Paper | N/A       | 0.777 ± 0.003 |
| 7    | ColorRefinement + Weighted Ensemble LGBM | Parker Burchett | GitHub, Paper | 68        | 0.767 ± 0.003 |
| 8    | Chemprop                                 | Kyle Swanson    | GitHub, Paper | N/A       | 0.754 ± 0.002 |
| 9    | AttentiveFP                              | Kexin Huang     | GitHub, Paper | 300,806   | 0.749 ± 0.004 |
| 10   | RDKit2D + MLP (DeepPurpose)              | Kexin Huang     | GitHub, Paper | 633,409   | 0.742 ± 0.006 |
| 11   | NeuralFP                                 | Kexin Huang     | GitHub, Paper | 480,193   | 0.739 ± 0.010 |
| 12   | GCN                                      | Kexin Huang     | GitHub, Paper | 191,810   | 0.735 ± 0.004 |
| 13   | Morgan + MLP (DeepPurpose)               | Kexin Huang     | GitHub, Paper | 1,477,185 | 0.715 ± 0.004 |
| 14   | CNN (DeepPurpose)                        | Kexin Huang     | GitHub, Paper | 226,625   | 0.713 ± 0.006 |
| 15   | Basic ML                                 | Nilavo Boral    | GitHub, Paper | N/A       | 0.556 ± 0.000 |
| 16   | Euclia ML model                          | Euclia          | GitHub, Paper | 50        | 0.536 ± 0.003 |

In [1]:
import pandas as pd
from deepmol.pipeline import Pipeline

2023-11-08 15:21:28.049099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 15:21:28.222142: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 15:21:28.223489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 15:21:29.053915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


In [2]:
# read results
results = pd.read_csv('cyp2c9/tdc_test_set_results.txt', sep=',', header=None, dtype={0: int, 1: float, 2: float})
# set columns
results.columns = ['trial_id', 'mean', 'std']
results

FileNotFoundError: [Errno 2] No such file or directory: 'cyp2c9/tdc_test_set_results.txt'

In [3]:
# order results by mean (std in case of tie)
results = results.sort_values(by=['mean', 'std'], ascending=False)
results

NameError: name 'results' is not defined

In [4]:
# load best trial pipeline (rank #...)
best_trial_id = int(results.iloc[0]['trial_id'])
pipeline = Pipeline.load(f"cyp2c9/trial_{best_trial_id}/")

NameError: name 'results' is not defined